In [ ]:
#INSTALAÇÃO DA BILIOTECA PYSPARK
!pip install pyspark
!pip install gcsfs

In [7]:
# Montando o Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#CHAMAR AS BIBLIOTECAS/MÓDULOS NECESSÁRIAS
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, FloatType
 
from pyspark import SparkConf
import pyspark.sql.functions as F

#CHAMAR AS BIBLIOTECAS/MÓDULOS NECESSÁRIAS
from google.cloud import storage
import os

import pandas as pd
!pip install pyarrow

#CONFIGURAÇÃO DA CHAVE DE SEGURANÇA
serviceAccount = '/content/drive/MyDrive/projeto_final/weighty-archive-349117-006e0d6bc25e.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#CÓDIGO QUE ACESSA A BUCKET CRIADA E FAZ O DOWNLOAD DOS ARQUIVOS VIA PYSPARK
#client = storage.Client()

#CRIAR VARIÁVEL PARA RECEBER O NOME DA BUCKET
#bucket = client.get_bucket('tratamentospark')

#USA O MÉTODO BLOB PARA RETORNAR O NOME DO ARQUIVO
#bucket.blob('netflix.csv')

#CRIA A VARIÁVEL PATH COLOCANDO O CAMINHO DE URI
#path = 'https://drive.google.com/drive/folders/1mTWORVc-KsRiBzb5FpBfR3b4f5Xz2iBw?usp=sharing'

#parDF=spark.read.parquet(path)

In [4]:
#CRIAR A SPARK SESSION E LER O ARQUIVO VIA PYSPARK
spark = (
    SparkSession.builder
                .master('local')
                .appName('spark-gcs')
                .config('spark.ui.port', '4050')
                .config("spark.jars", 'https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop2-latest.jar')
                .getOrCreate()
)

In [5]:
#Formando a Estrutura do DataFrame em pySpark
esquema = (
    StructType([                
        StructField('Cod_Empresa', StringType(), True),
        StructField('Empresa', StringType(), True),
        StructField('Dt_Negociacao', DateType(), False),
        StructField('Abertura', FloatType(), True),                
        StructField('Max', FloatType(), True),
        StructField('Min', FloatType(), True),
        StructField('Fechamento', FloatType(), True),
        StructField('Volume_Negociado', FloatType(), True),
        StructField('Valor_Mercado_Empresa', FloatType(), True) 
    ])
)


In [8]:
# Download do DataSet CSV do GCP/Bucket, e Cria DataFrame com StructType do pySpark
df_spark = (
    spark.read.format('csv')
              .option('header', True)
              .option('delimiter', ',')
              .option('inferschema', True)
              .load("gs://cryptocurrencydb/Tratados/Tratado_SparkValorMercadoMaiorZero_2022_06_09_Aldreks.csv", schema=esquema))

In [ ]:
df_spark.printSchema()

In [9]:
df_spark.show()

+-----------+---------+-------------+--------+--------+--------+----------+----------------+---------------------+
|Cod_Empresa|  Empresa|Dt_Negociacao|Abertura|     Max|     Min|Fechamento|Volume_Negociado|Valor_Mercado_Empresa|
+-----------+---------+-------------+--------+--------+--------+----------+----------------+---------------------+
|        ANC| Anoncoin|   2013-12-27|    5.21|    5.39|    4.64|      4.81|         16203.0|            2975246.0|
|        BTC|  Bitcoin|   2013-12-27|  763.28|  777.51|   713.6|    735.07|       4.68627E7|          8.9553951E9|
|        DMD|  Diamond|   2013-12-27|    2.29|     2.5|    1.49|      1.75|          8360.0|             341344.0|
|       DOGE| Dogecoin|   2013-12-27| 6.03E-4|6.282E-4|4.969E-4|  5.219E-4|        477422.0|            8016604.0|
|        LTC| Litecoin|   2013-12-27|   24.81|   25.27|   22.26|     23.27|       3.11122E7|          5.6608806E8|
|        TRC|Terracoin|   2013-12-27|  0.5038|  0.5251|    0.45|    0.4778|     

In [10]:
# Insight: Análise de volume negociado por ano das criptos

# criando a coluna ano para fazer a agregação 
df_spark_01 = df_spark.select(F.col('Dt_Negociacao'), F.year(F.col('Dt_Negociacao'))).alias('ano').show()

#df_spark_01 = df_spark.withColumn("ano",  F.year(F.col('Dt_Negociacao')))
#df_spark_01.show()
# agregação volume negociado
 
#df_spark_01.groupBy("ano").count().show(n=10)
#df_spark_02 = df_spark_01.groupBy("ano").agg(F.sum("Volume$_Negociado")).show(n=20)
#df_spark_02 = df_spark_01.select(F.col("ano")).groupBy("ano").agg(F.sum("Volume$_Negociado")).show(n=20)

#len(df_spark_02)


+-------------+-------------------+
|Dt_Negociacao|year(Dt_Negociacao)|
+-------------+-------------------+
|   2013-12-27|               2013|
|   2013-12-27|               2013|
|   2013-12-27|               2013|
|   2013-12-27|               2013|
|   2013-12-27|               2013|
|   2013-12-27|               2013|
|   2013-12-27|               2013|
|   2013-12-28|               2013|
|   2013-12-28|               2013|
|   2013-12-28|               2013|
|   2013-12-28|               2013|
|   2013-12-28|               2013|
|   2013-12-28|               2013|
|   2013-12-28|               2013|
|   2013-12-29|               2013|
|   2013-12-29|               2013|
|   2013-12-29|               2013|
|   2013-12-29|               2013|
|   2013-12-29|               2013|
|   2013-12-29|               2013|
+-------------+-------------------+
only showing top 20 rows



In [11]:
# criando a coluna de ano de negociação na criptomoeada segundo o banco de dados da coinmarketcap.com
df_spark_01 = df_spark.withColumn("ano",  F.year(F.col('Dt_Negociacao')))

In [12]:
df_spark_01.show()

+-----------+---------+-------------+--------+--------+--------+----------+----------------+---------------------+----+
|Cod_Empresa|  Empresa|Dt_Negociacao|Abertura|     Max|     Min|Fechamento|Volume_Negociado|Valor_Mercado_Empresa| ano|
+-----------+---------+-------------+--------+--------+--------+----------+----------------+---------------------+----+
|        ANC| Anoncoin|   2013-12-27|    5.21|    5.39|    4.64|      4.81|         16203.0|            2975246.0|2013|
|        BTC|  Bitcoin|   2013-12-27|  763.28|  777.51|   713.6|    735.07|       4.68627E7|          8.9553951E9|2013|
|        DMD|  Diamond|   2013-12-27|    2.29|     2.5|    1.49|      1.75|          8360.0|             341344.0|2013|
|       DOGE| Dogecoin|   2013-12-27| 6.03E-4|6.282E-4|4.969E-4|  5.219E-4|        477422.0|            8016604.0|2013|
|        LTC| Litecoin|   2013-12-27|   24.81|   25.27|   22.26|     23.27|       3.11122E7|          5.6608806E8|2013|
|        TRC|Terracoin|   2013-12-27|  0

In [13]:
# somando o volume negociado por ano as criptos
df_spark_02 = df_spark_01.groupBy("ano").agg(F.sum("Volume_Negociado")).show(n=20)

+----+---------------------+
| ano|sum(Volume_Negociado)|
+----+---------------------+
|2018|    6.653449277567E12|
|2015|      1.4244437163E10|
|2022|   2.0534279612546E13|
|2013|         2.15792568E8|
|2014|       1.222451483E10|
|2019|    2.034094881157E13|
|2020|   4.4710202158703E13|
|2016|      4.3725791794E10|
|2017|    2.040412773322E12|
|2021|   9.5933759181702E13|
+----+---------------------+



In [23]:
df_spark_02 = df_spark_01.select(F.col("ano"), F.col("Volume_Negociado")).groupBy("ano").agg(F.sum("Volume_Negociado"))

In [21]:
df_spark_02.show()

+----+---------------------+
| ano|sum(Volume_Negociado)|
+----+---------------------+
|2018|    6.653449277567E12|
|2015|      1.4244437163E10|
|2022|   2.0534279612546E13|
|2013|         2.15792568E8|
|2014|       1.222451483E10|
|2019|    2.034094881157E13|
|2020|   4.4710202158703E13|
|2016|      4.3725791794E10|
|2017|    2.040412773322E12|
|2021|   9.5933759181702E13|
+----+---------------------+



In [22]:
# gravando o df volume negociado por ano em csv

df_spark_02.write.csv("gs://cryptocurrencydb/Tratados/volume_negociado_ano")